<img src="media/Cryo+ESA_Logo_plain-50gray.svg" alt="CCI Cryosphere banner" width="1000"/>

# Greenland mass change exercise using the Open Data Portal

The Gravity Recovery and Climate Experiment (GRACE, and it's follow-on GRACE-FO), [have the unique ability to 'weigh' the ice sheets from space](https://earth.esa.int/eogateway/missions/grace). The satellite mission uses the principle that satellite orbits are affected by gravitational changes of the Earth. In reversal, a very accurate tracking of the satellite orbits over time allows the determination of mass changes on the Earth.

In this exercise we download and visualize a GRACE CCI product of ice mass change using cate. After the exercise you'll have an idea of the current imbalance of the Greenland icesheet and see where the icesheet is losing mass.

## Preparations
If you haven't done so please follow the [Cate tutorial](futurelearn.com/tbd) to get started using the exercises.


In [ ]:
# To get things started we need to initialize a few things
#Load some python modules to make them accessible to the notebook
from cate.core.ds import DATA_STORE_POOL
import cate.ops as ops
from cate.util.monitor import ConsoleMonitor
from cate.core.ds import get_metadata_from_descriptor
from cate.ops.io import open_dataset

# the following is needed to run Cate in a Jupyter Notebook
from xcube.util.ipython import enable_asyncio
enable_asyncio()

# utilities
from IPython.display import display
import numpy as np
from datetime import datetime

monitor=ConsoleMonitor()

To begin, let us see which data stores are available in the Data Store Pool.

In [ ]:
DATA_STORE_POOL.store_instance_ids

We see three stores. The 'cci-store' is a store that provides access to all datasets from the CCI Open Data Portal. The 'cci-zarr-store' is a store that contains selected data from the Open Data Portal, converted to the zarr format. The datasets from this store can be opened and processed faster, but the store provides only a small subset of what is offered by the 'cci-store'. The 'local' data store finally allows to access locally provided data. Also, when you select to cache data, you will find it in this store. Cached data can also be opened quickly.

For starters, let us use the 'cci-store' which connects to the Open Data Portal and allows access to the datasets listed at https://catalogue.ceda.ac.uk. We can ask for the search parameters of the store to see how a search can be performed.

In [ ]:
data_store = DATA_STORE_POOL.get_store('cci-store')
data_store.get_search_params_schema()

For this exercise, we are interested in GRACE data sets, so we ask for datasets of the platform "GRACE". We then display the results of the query.

In [ ]:
# lets query the dataserver
icesheetqry=data_store.search_data(platform="GRACE")

for i,item in enumerate(icesheetqry):
    print(f'\nDataset item {i}:')
    display(item)

Of these, we pick "esacci.ICESHEETS.yr.Unspecified.GMB.GRACE-instrument.GRACE.UNSPECIFIED.1-4.greenland_gmb_mass_trends" and open it. The parameter 'data_store_id' is not absolutely necessary, but it makes the opening a little faster.

In [ ]:
greenTrnd=open_dataset(ds_id="esacci.ICESHEETS.yr.Unspecified.GMB.GRACE-instrument.GRACE.UNSPECIFIED.1-4.greenland_gmb_mass_trends",
                       data_store_id='cci-store')

## Create a scatter pot of the mass trends

In [ ]:
from xarray.plot import scatter
import cartopy.crs as ccrs
import matplotlib.pyplot as mpl
import pandas as pd
import numpy as np

%matplotlib inline
itime=0
yrstart=np.datetime_as_string(greenTrnd.start_time[itime],unit="Y")
yrend=np.datetime_as_string(greenTrnd.end_time[itime],unit="Y")
mpl.figure(figsize=(16,8))
crs=ccrs.NorthPolarStereo(-45)
xy=crs.transform_points(ccrs.PlateCarree(),greenTrnd.longitude.values,greenTrnd.latitude.values)
ax = mpl.subplot(projection=crs)
ax.coastlines()
greenTrnd.isel(time=itime).plot.scatter(x="longitude",y="latitude",hue="GMB_trend",cmap="RdBu",transform=ccrs.PlateCarree(),ax=ax,vmin=-500,vmax=500,s=70,marker='H')
ax.set_title("mm/yr (equivalent water) %s - %s"%(yrstart,yrend))

## Questions
* Try to select different trend periods (variable `itime`). In which regions do you see changes?
* To better visualize the areas which have undergone change, adapt (or copy) the code and plot the difference between two trend periods.
* The spatial resolution of mass changes from GRACE and GRACE-FO are much coarser than what can be obtained from for example ice altimetry from e.g. Cryosat-2, but there are advantages too. Try to make a list of advantages/disadvantages of gravimetry versus altimetry and exchange your findings ont he forum.
